In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.28 🚀 Python-3.8.19 torch-2.2.2 CPU (Intel Core(TM) i7-3520M 2.90GHz)
Setup complete ✅ (4 CPUs, 8.0 GB RAM, 101.3/465.6 GB disk)


In [2]:
import cv2
from ultralytics import YOLO
import numpy as np
import time
import pandas as pd

In [3]:
# Load model and mask
model = YOLO("/Users/macbook/Pengujian/videos_models/yolov8l.pt")
mask = cv2.imread("/Users/macbook/Pengujian/datasets_test/mask.jpg", cv2.IMREAD_GRAYSCALE)

In [4]:
# Class names for YOLOv8 model (adjust according to your model's classes)
class_names = {0: "car", 1: "motorcycle", 2: "person"}

# Function to draw bounding boxes with different colors for each class
def draw_boxes(result, frame):
    blank = np.zeros(frame.shape, dtype=np.uint8)
    image = frame.copy()
    for box in result.boxes:
        x1, y1, x2, y2 = [round(x) for x in box.xyxy[0].tolist()]
        conf = box.conf[0]  # Confidence score
        cls = int(box.cls[0])  # Class label
        
        # Assign colors based on class
        if cls == 0:  # car
            color = (0, 255, 0)  # Green
        elif cls == 1:  # motorcycle
            color = (0, 0, 255)  # Red
        elif cls == 2:  # person
            color = (255, 0, 0)  # Blue
        else:
            color = (255, 255, 255)  # Default to white if class not recognized
        
        label = f"{class_names.get(cls, 'unknown')}: {conf:.2f}"
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.rectangle(blank, (x1, y1), (x2, y2), (255, 255, 255), -1)
    blank = cv2.bitwise_and(blank, blank, mask=mask)
    blank = cv2.cvtColor(blank, cv2.COLOR_BGR2GRAY)
    return blank, image

# Detection function
def detect(frame, threshold=30):
    masked = cv2.bitwise_and(frame, frame, mask=mask)
    result = model(masked, conf=0.25, verbose=False)
    area, image = draw_boxes(result[0], frame)
    percentage = round((np.sum(area) / np.sum(mask)) * 100, 2)
    label = "ramai" if percentage > threshold else "tidak ramai"
    return area, percentage, label, image

# Video processing function
def process_video(video_path, threshold=30, log_path="yolov8l_log_data.csv"):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter("/Users/macbook/Pengujian/videos/yolov8l_videos.mp4",
                          cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    start_time = time.time()

    # List to store log data
    log_data = []

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        area, percentage, kondisi, processed_frame = detect(frame, threshold)
        elapsed_time = time.time() - start_time

        # Overlay text information on frame
        cv2.rectangle(processed_frame, (0, 0), (200, 60), (0, 0, 0), -1)
        cv2.putText(processed_frame, f"Area: {percentage}%", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(processed_frame, f"FPS: {1 / elapsed_time:.2f}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        cv2.putText(processed_frame, f"Kondisi: {kondisi}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        out.write(processed_frame)
        start_time = time.time()

        # Store log data
        log_data.append([frame_count, elapsed_time, percentage, kondisi])

        # Display frame
        cv2.imshow("Video", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Save log data to CSV
    log_df = pd.DataFrame(log_data, columns=["Frame", "Elapsed Time (s)", "Area (%)", "Kondisi"])
    log_df.to_csv(log_path, index=False)

In [5]:
# Run the video processing function on your video file
video_path = "/Users/macbook/Pengujian/videos/videos.mp4"
process_video(video_path, threshold=30, log_path="/Users/macbook/Pengujian/videos_result/yolov8l_log_data.csv")

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
